In [1]:
! pip3 install confluent_kafka

    100% |################################| 7.4MB 134kB/s ta 0:00:01


In [2]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
import logging

logging.basicConfig()

a = AdminClient({'bootstrap.servers': 'localhost:9092,localhost:9093,localhost:9094'})

In [3]:
def create_topics(a, topics):
    """ Create topics """

    new_topics = [NewTopic(topic, num_partitions=1, replication_factor=2) for topic in topics]
    # Call create_topics to asynchronously create topics, a dict
    # of <topic,future> is returned.
    fs = a.create_topics(new_topics)

    # Wait for operation to finish.
    # Timeouts are preferably controlled by passing request_timeout=15.0
    # to the create_topics() call.
    # All futures will finish at the same time.
    for topic, f in fs.items():
        try:
            f.result()  # The result itself is None
            print("Topic {} created".format(topic))
        except Exception as e:
            print("Failed to create topic {}: {}".format(topic, e))

In [9]:
topic_names = ["ticker","orderbook","trade"]

create_topics(a,topics=topic_names)

Topic ticker created
Failed to create topic orderbook: KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'orderbook' already exists."}
Failed to create topic trade: KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'trade' already exists."}


In [10]:
def list_topic_metadata(a, arg = "all"):
    """ list topics and cluster metadata """

    what = arg

    md = a.list_topics(timeout=10)

    print("Cluster {} metadata (response from broker {}):".format(md.cluster_id, md.orig_broker_name))

    if what in ("all", "brokers"):
        print(" {} brokers:".format(len(md.brokers)))
        for b in iter(md.brokers.values()):
            if b.id == md.controller_id:
                print("  {}  (controller)".format(b))
            else:
                print("  {}".format(b))

    if what not in ("all", "topics"):
        return

    print(" {} topics:".format(len(md.topics)))
    for t in iter(md.topics.values()):
        if t.error is not None:
            errstr = ": {}".format(t.error)
        else:
            errstr = ""

        print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

        for p in iter(t.partitions.values()):
            if p.error is not None:
                errstr = ": {}".format(p.error)
            else:
                errstr = ""

            print("    partition {} leader: {}, replicas: {}, isrs: {}".format(
                p.id, p.leader, p.replicas, p.isrs, errstr))

In [13]:
list_topic_metadata(a,"all")

Cluster VEOYyBtnR8-8ccJbGjaR_Q metadata (response from broker 127.0.0.1:9092/1):
 3 brokers:
  127.0.0.1:9093/2
  127.0.0.1:9094/3
  127.0.0.1:9092/1  (controller)
 6 topics:
  "trade" with 1 partition(s)
    partition 0 leader: 2, replicas: [2, 3], isrs: [2, 3]
  "__consumer_offsets" with 50 partition(s)
    partition 0 leader: 3, replicas: [3, 1, 2], isrs: [3, 1, 2]
    partition 1 leader: 1, replicas: [1, 2, 3], isrs: [1, 2, 3]
    partition 2 leader: 2, replicas: [2, 3, 1], isrs: [2, 3, 1]
    partition 3 leader: 3, replicas: [3, 2, 1], isrs: [3, 2, 1]
    partition 4 leader: 1, replicas: [1, 3, 2], isrs: [1, 3, 2]
    partition 5 leader: 2, replicas: [2, 1, 3], isrs: [2, 1, 3]
    partition 6 leader: 3, replicas: [3, 1, 2], isrs: [3, 1, 2]
    partition 7 leader: 1, replicas: [1, 2, 3], isrs: [1, 2, 3]
    partition 8 leader: 2, replicas: [2, 3, 1], isrs: [2, 3, 1]
    partition 9 leader: 3, replicas: [3, 2, 1], isrs: [3, 2, 1]
    partition 10 leader: 1, replicas: [1, 3, 2], isrs